In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import collections
import scipy
import operator
from functools import reduce
import community as cmt
from networkx.algorithms.community import greedy_modularity_communities
import seaborn as sns
from sklearn.mixture import GaussianMixture as GMM
import random
sns.set(rc={'figure.figsize':(16,9)})
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
routes = pd.read_csv('routes.dat', header=None,na_values='\\N')
routes.columns = ['Airline',
                'Airline ID',
                'Source airport',
                'Source airport ID',
                'Destination airport',
                'Destination airport ID',
                'Codeshare',
                'Stops',
                'Equipment']

airports = pd.read_csv('airports.dat', header=None, index_col=4, na_values='\\N')
airports.columns = ['Airport ID',
                    'Name',
                    'City',
                    'Country',
                    'ICAO',
                    'Latitude',
                    'Longitude',
                    'Altitude',
                    'Timezone',
                    'DST',
                    'TZ',
                    'Type',
                    'Source']

In [4]:
Asean = ['Vietnam','Thailand','Singapore','Laos',
         'Burma','Cambodia','Malaysia','Indonesia','Philippines','Brunei']
airports_asean = airports[airports['Country'].isin(Asean)]
airports_asean.dropna()
air_asean = list(airports_asean.index)
airports_route = list(set(set(routes['Source airport']) | set(routes['Destination airport'])))
nodes_asean = list(set(air_asean) & set(airports_route))
routes_asean = routes[(routes['Source airport'].isin(nodes_asean)) & (routes['Destination airport'].isin(nodes_asean))]
airports_asean = airports[airports.index.isin(nodes_asean)]

In [5]:
#We add the lat.long for starting and arriving airports and the distance of the route(5 columns)
routes_asean['Source latitude'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Source airport ID'])].Latitude.values.item(0), axis = 1)
routes_asean['Source longitude'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Source airport ID'])].Longitude.values.item(0), axis = 1)
routes_asean['Destination latitude'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Destination airport ID'])].Latitude.values.item(0), axis = 1)
routes_asean['Destination longitude'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Destination airport ID'])].Longitude.values.item(0), axis = 1)
routes_asean['Distance'] = routes_asean.apply(lambda row : geodesic((row['Source latitude'], row['Source longitude']), (row['Destination latitude'], row['Destination longitude'])).kilometers, axis = 1)

In [6]:
airports_asean.head()

,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source
4,,,,,,,,,,,,,
MNL,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.508600,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports
CBO,2399,Awang Airport,Cotabato,Philippines,RPMC,7.165240,124.209999,189,8.0,N,Asia/Manila,airport,OurAirports
CGY,2400,Cagayan De Oro Airport,Ladag,Philippines,RPML,8.415620,124.611000,601,8.0,N,Asia/Manila,airport,OurAirports
PAG,2401,Pagadian Airport,Pagadian,Philippines,RPMP,7.830731,123.461180,5,8.0,N,Asia/Manila,airport,OurAirports
GES,2402,General Santos International Airport,Romblon,Philippines,RPMR,6.058000,125.096001,505,8.0,N,Asia/Manila,airport,OurAirports


In [7]:
#we can add attributes here
G = nx.from_pandas_edgelist(routes_asean, 'Source airport', 'Destination airport', ['Distance'])

In [8]:
deg_distribution = dict(G.degree())

In [9]:
k = list(deg_distribution.values())

In [10]:
air = list(deg_distribution.keys())
deg = list(deg_distribution.values())

In [11]:
betweenness = nx.betweenness_centrality(G)

In [12]:
degdistr = {'APT':air,'Deg':deg}
deg_df = pd.DataFrame.from_dict(degdistr)
deg_df.shape

(210, 2)

In [13]:
bet_air = list(betweenness.keys())
bet = list(betweenness.values())
between = {'APT':bet_air,'BET':bet}
betdf = pd.DataFrame.from_dict(between)
betdf.shape

(210, 2)

In [14]:
totaldf = pd.merge(deg_df,betdf,on='APT')

In [16]:
air_cp = airports_asean
air_cp['APT'] = air_cp.index

In [17]:
air_cp = air_cp.reset_index(drop=True)

In [18]:
air_cp.head()

,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source,APT
0,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.508600,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports,MNL
1,2399,Awang Airport,Cotabato,Philippines,RPMC,7.165240,124.209999,189,8.0,N,Asia/Manila,airport,OurAirports,CBO
2,2400,Cagayan De Oro Airport,Ladag,Philippines,RPML,8.415620,124.611000,601,8.0,N,Asia/Manila,airport,OurAirports,CGY
3,2401,Pagadian Airport,Pagadian,Philippines,RPMP,7.830731,123.461180,5,8.0,N,Asia/Manila,airport,OurAirports,PAG
4,2402,General Santos International Airport,Romblon,Philippines,RPMR,6.058000,125.096001,505,8.0,N,Asia/Manila,airport,OurAirports,GES


In [19]:
totaldf.head()

,APT,Deg,BET
0,BSO,1,0.000000
1,MNL,44,0.244047
2,BXU,2,0.000000
3,CBO,1,0.000000
4,CGY,5,0.000012


In [20]:
totaldf = pd.merge(totaldf,air_cp,on = 'APT')
totaldf.sort_values(by='Deg',ascending=False).head(30)

,APT,Deg,BET,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source
20,KUL,48,0.176914,3304,Kuala Lumpur International Airport,Kuala Lumpur,Malaysia,WMKK,2.745580,101.709999,69,8.0,N,Asia/Kuala_Lumpur,airport,OurAirports
18,SIN,46,0.196338,3316,Singapore Changi Airport,Singapore,Singapore,WSSS,1.350190,103.994003,22,8.0,N,Asia/Singapore,airport,OurAirports
1,MNL,44,0.244047,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.508600,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports
39,CGK,42,0.210816,3275,Soekarno-Hatta International Airport,Jakarta,Indonesia,WIII,-6.125570,106.655998,34,7.0,N,Asia/Jakarta,airport,OurAirports
72,DMK,35,0.153754,3157,Don Mueang International Airport,Bangkok,Thailand,VTBD,13.912600,100.607002,9,7.0,U,Asia/Bangkok,airport,OurAirports
17,BKK,31,0.098338,3885,Suvarnabhumi Airport,Bangkok,Thailand,VTBS,13.681100,100.747002,5,7.0,U,Asia/Bangkok,airport,OurAirports
25,SGN,26,0.101577,3205,Tan Son Nhat International Airport,Ho Chi Minh City,Vietnam,VVTS,10.818800,106.652000,33,7.0,U,Asia/Saigon,airport,OurAirports
42,HAN,24,0.064193,3199,Noi Bai International Airport,Hanoi,Vietnam,VVNB,21.221201,105.806999,39,7.0,U,Asia/Saigon,airport,OurAirports
27,CEB,24,0.041576,4206,Mactan Cebu International Airport,Cebu,Philippines,RPVM,10.307500,123.978996,31,8.0,N,Asia/Manila,airport,OurAirports
41,DPS,24,0.079898,3940,Ngurah Rai (Bali) International Airport,Denpasar,Indonesia,WADD,-8.748170,115.167000,14,8.0,N,Asia/Makassar,airport,OurAirports


In [21]:
eigenCenDict = nx.eigenvector_centrality(G)
eigenAir = list(eigenCenDict.keys())
eigenCen = list(eigenCenDict.values())

In [22]:
eigenCentrality = {'APT':eigenAir,'Eigen':eigenCen}
eigendf = pd.DataFrame.from_dict(eigenCentrality)
eigendf.head()

,APT,Eigen
0,BSO,0.012685
1,MNL,0.196200
2,BXU,0.018670
3,CBO,0.012685
4,CGY,0.026798


In [23]:
totaldf = pd.merge(totaldf,eigendf,on = 'APT')
totaldf.head()

,APT,Deg,BET,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source,Eigen
0,BSO,1,0.000000,6019,Basco Airport,Basco,Philippines,RPUO,20.45130,121.980003,291,8.0,N,NaN,airport,OurAirports,0.012685
1,MNL,44,0.244047,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.50860,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports,0.196200
2,BXU,2,0.000000,4200,Bancasi Airport,Butuan,Philippines,RPME,8.95150,125.478800,141,8.0,N,Asia/Manila,airport,OurAirports,0.018670
3,CBO,1,0.000000,2399,Awang Airport,Cotabato,Philippines,RPMC,7.16524,124.209999,189,8.0,N,Asia/Manila,airport,OurAirports,0.012685
4,CGY,5,0.000012,2400,Cagayan De Oro Airport,Ladag,Philippines,RPML,8.41562,124.611000,601,8.0,N,Asia/Manila,airport,OurAirports,0.026798


In [24]:
ccraw = nx.clustering(G)
ccAir = list(ccraw.keys())
cc = list(ccraw.values())

In [25]:
ccDict = {'APT':ccAir,'CC':cc}
ccdf = pd.DataFrame.from_dict(ccDict)
ccdf.head()

,APT,CC
0,BSO,0.000000
1,MNL,0.078224
2,BXU,1.000000
3,CBO,0.000000
4,CGY,0.900000


In [26]:
totaldf = pd.merge(totaldf,ccdf,on = 'APT')
totaldf.head()

,APT,Deg,BET,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source,Eigen,CC
0,BSO,1,0.000000,6019,Basco Airport,Basco,Philippines,RPUO,20.45130,121.980003,291,8.0,N,NaN,airport,OurAirports,0.012685,0.000000
1,MNL,44,0.244047,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.50860,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports,0.196200,0.078224
2,BXU,2,0.000000,4200,Bancasi Airport,Butuan,Philippines,RPME,8.95150,125.478800,141,8.0,N,Asia/Manila,airport,OurAirports,0.018670,1.000000
3,CBO,1,0.000000,2399,Awang Airport,Cotabato,Philippines,RPMC,7.16524,124.209999,189,8.0,N,Asia/Manila,airport,OurAirports,0.012685,0.000000
4,CGY,5,0.000012,2400,Cagayan De Oro Airport,Ladag,Philippines,RPML,8.41562,124.611000,601,8.0,N,Asia/Manila,airport,OurAirports,0.026798,0.900000


In [27]:
hub,authority = nx.hits(G)

In [28]:
hubAir = list(hub.keys())
hubVal = list(hub.values())
hubDict = {'APT':hubAir,'HUBAUTH':hubVal}
hubdf = pd.DataFrame.from_dict(hubDict)
hubdf.head()

,APT,HUBAUTH
0,BSO,0.001466
1,MNL,0.022679
2,BXU,0.002158
3,CBO,0.001466
4,CGY,0.003097


In [29]:
totaldf = pd.merge(totaldf,hubdf,on = 'APT')
totaldf.head()

,APT,Deg,BET,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source,Eigen,CC,HUBAUTH
0,BSO,1,0.000000,6019,Basco Airport,Basco,Philippines,RPUO,20.45130,121.980003,291,8.0,N,NaN,airport,OurAirports,0.012685,0.000000,0.001466
1,MNL,44,0.244047,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.50860,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports,0.196200,0.078224,0.022679
2,BXU,2,0.000000,4200,Bancasi Airport,Butuan,Philippines,RPME,8.95150,125.478800,141,8.0,N,Asia/Manila,airport,OurAirports,0.018670,1.000000,0.002158
3,CBO,1,0.000000,2399,Awang Airport,Cotabato,Philippines,RPMC,7.16524,124.209999,189,8.0,N,Asia/Manila,airport,OurAirports,0.012685,0.000000,0.001466
4,CGY,5,0.000012,2400,Cagayan De Oro Airport,Ladag,Philippines,RPML,8.41562,124.611000,601,8.0,N,Asia/Manila,airport,OurAirports,0.026798,0.900000,0.003097


In [30]:
pagerankraw= nx.pagerank(G)
pagerankAir = list(pagerankraw.keys())
pagerankVal = list(pagerankraw.values())
pagerankDict = {'APT':pagerankAir,'PAGERANK':pagerankVal}
pagerankdf = pd.DataFrame.from_dict(pagerankDict)
pagerankdf.head()

,APT,PAGERANK
0,BSO,0.001532
1,MNL,0.042334
2,BXU,0.002313
3,CBO,0.001532
4,CGY,0.004598


In [31]:
totaldf = pd.merge(totaldf,pagerankdf,on = 'APT')
totaldf.head()

,APT,Deg,BET,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source,Eigen,CC,HUBAUTH,PAGERANK
0,BSO,1,0.000000,6019,Basco Airport,Basco,Philippines,RPUO,20.45130,121.980003,291,8.0,N,NaN,airport,OurAirports,0.012685,0.000000,0.001466,0.001532
1,MNL,44,0.244047,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.50860,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports,0.196200,0.078224,0.022679,0.042334
2,BXU,2,0.000000,4200,Bancasi Airport,Butuan,Philippines,RPME,8.95150,125.478800,141,8.0,N,Asia/Manila,airport,OurAirports,0.018670,1.000000,0.002158,0.002313
3,CBO,1,0.000000,2399,Awang Airport,Cotabato,Philippines,RPMC,7.16524,124.209999,189,8.0,N,Asia/Manila,airport,OurAirports,0.012685,0.000000,0.001466,0.001532
4,CGY,5,0.000012,2400,Cagayan De Oro Airport,Ladag,Philippines,RPML,8.41562,124.611000,601,8.0,N,Asia/Manila,airport,OurAirports,0.026798,0.900000,0.003097,0.004598


In [32]:
c1 = nx.algorithms.community.greedy_modularity_communities(G)

In [33]:
j=1
[j+2]*2

[3, 3]

In [34]:
for i in range(len(c1)):
    if(i==0):
        tempAir = list(c1[i])
        tempComm = [i+1]*len(tempAir)
        tempDict = {'APT':tempAir,'CommGM':tempComm}
        commdf = pd.DataFrame.from_dict(tempDict)
    else:
        tempAir = list(c1[i])
        tempComm = [i+1]*len(tempAir)
        tempDict = {'APT':tempAir,'CommGM':tempComm}
        tempdf = pd.DataFrame.from_dict(tempDict)
        commdf = commdf.append(tempdf)

In [35]:
commdf.shape

(210, 2)

In [36]:
from community import community_louvain
partition = community_louvain.best_partition(G)

In [37]:
LVAir = list(partition.keys())
commLV = list(partition.values())
dictLV = {'APT':LVAir,'CommLV':commLV}
LVdf = pd.DataFrame.from_dict(dictLV)
commdf = pd.merge(commdf,LVdf,on='APT')

In [38]:
commdf.shape

(210, 3)

In [39]:
commdf.head()

,APT,CommGM,CommLV
0,NAH,1,6
1,TIM,1,6
2,TJQ,1,5
3,JOG,1,5
4,DJB,1,5


In [40]:
nx.algorithms.community.quality.coverage(G,c1)

0.7673179396092362

In [41]:
nx.algorithms.community.quality.modularity(G,c1)

0.5583511321296118

In [42]:
totaldf = pd.merge(totaldf,commdf,on='APT')
totaldf.to_csv("statAsean.csv")

In [43]:
routes_asean.to_csv("routesAsean.csv")

In [33]:
Asean = ['Vietnam','Thailand','Singapore','Laos',
         'Burma','Cambodia','Malaysia','Indonesia','Philippines','Brunei']
airports_asean = airports[airports['Country'].isin(Asean)]

,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source
4,,,,,,,,,,,,,
MNL,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.508600,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports
CBO,2399,Awang Airport,Cotabato,Philippines,RPMC,7.165240,124.209999,189,8.0,N,Asia/Manila,airport,OurAirports
CGY,2400,Cagayan De Oro Airport,Ladag,Philippines,RPML,8.415620,124.611000,601,8.0,N,Asia/Manila,airport,OurAirports
PAG,2401,Pagadian Airport,Pagadian,Philippines,RPMP,7.830731,123.461180,5,8.0,N,Asia/Manila,airport,OurAirports
GES,2402,General Santos International Airport,Romblon,Philippines,RPMR,6.058000,125.096001,505,8.0,N,Asia/Manila,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...
AAV,11926,Allah Valley Airport,Surallah,Philippines,RPMA,6.366820,124.750999,659,NaN,NaN,NaN,airport,OurAirports
BPH,11927,Bislig Airport,NaN,Philippines,RPMF,8.195950,126.321999,12,NaN,NaN,NaN,airport,OurAirports
MXI,11928,Mati National Airport,NaN,Philippines,RPMQ,6.949370,126.273003,156,NaN,NaN,NaN,airport,OurAirports


In [35]:
airports_asean.dropna()

,Airport ID,Name,City,Country,ICAO,Latitude,Longitude,Altitude,Timezone,DST,TZ,Type,Source
4,,,,,,,,,,,,,
MNL,2397,Ninoy Aquino International Airport,Manila,Philippines,RPLL,14.508600,121.019997,75,8.0,N,Asia/Manila,airport,OurAirports
CBO,2399,Awang Airport,Cotabato,Philippines,RPMC,7.165240,124.209999,189,8.0,N,Asia/Manila,airport,OurAirports
CGY,2400,Cagayan De Oro Airport,Ladag,Philippines,RPML,8.415620,124.611000,601,8.0,N,Asia/Manila,airport,OurAirports
PAG,2401,Pagadian Airport,Pagadian,Philippines,RPMP,7.830731,123.461180,5,8.0,N,Asia/Manila,airport,OurAirports
GES,2402,General Santos International Airport,Romblon,Philippines,RPMR,6.058000,125.096001,505,8.0,N,Asia/Manila,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...
NTI,9872,Stenkol Airport,Bintuni,Indonesia,WASB,-2.103300,133.516400,57,9.0,N,Asia/Jayapura,airport,OurAirports
WSR,9873,Wasior Airport,Wasior,Indonesia,WASW,-2.721000,134.506100,49,9.0,N,Asia/Jayapura,airport,OurAirports
DTB,9874,Silangit Airport,Siborong-Borong,Indonesia,WIMN,2.259730,98.991898,4700,7.0,N,Asia/Jakarta,airport,OurAirports


In [38]:
air_asean = list(airports_asean.index)

True

In [43]:
airports_route = list(set(set(routes['Source airport']) | set(routes['Destination airport'])))
nodes_asean = list(set(air_asean) & set(airports_route))
routes_asean = routes[(routes['Source airport'].isin(nodes_asean)) & (routes['Destination airport'].isin(nodes_asean))]
airports_asean = airports[airports.index.isin(nodes_asean)]

In [44]:
routes_asean

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment
180,2P,897.0,BSO,6019.0,MNL,2397.0,NaN,0,DH4
181,2P,897.0,BXU,4200.0,MNL,2397.0,NaN,0,320
182,2P,897.0,CBO,2399.0,MNL,2397.0,NaN,0,320
183,2P,897.0,CGY,2400.0,MNL,2397.0,NaN,0,320
184,2P,897.0,CRM,6024.0,MNL,2397.0,NaN,0,DH4
...,...,...,...,...,...,...,...,...,...
66286,Z2,9764.0,MNL,2397.0,TAG,3994.0,NaN,0,320 319
66287,Z2,9764.0,MYY,3266.0,MNL,2397.0,NaN,0,320
66288,Z2,9764.0,PPS,2433.0,MNL,2397.0,NaN,0,320 319
66291,Z2,9764.0,TAC,2422.0,MNL,2397.0,NaN,0,320


In [46]:
#We add the lat.long for starting and arriving airports and the distance of the route(5 columns)
routes_asean['Source latitude'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Source airport ID'])].Latitude.values.item(0), axis = 1)
routes_asean['Source longitude'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Source airport ID'])].Longitude.values.item(0), axis = 1)
routes_asean['Source Country'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Source airport ID'])].Country.values.item(0), axis = 1)

routes_asean['Destination latitude'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Destination airport ID'])].Latitude.values.item(0), axis = 1)
routes_asean['Destination longitude'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Destination airport ID'])].Longitude.values.item(0), axis = 1)
routes_asean['Destination Country'] = routes_asean.apply(lambda row : airports_asean[airports_asean['Airport ID'] == int(row['Destination airport ID'])].Country.values.item(0), axis = 1)

routes_asean['Distance'] = routes_asean.apply(lambda row : geodesic((row['Source latitude'], row['Source longitude']), (row['Destination latitude'], row['Destination longitude'])).kilometers, axis = 1)

In [48]:
routes_asean

,Airline,Airline ID,Source airport,Source airport ID,Destination airport,Destination airport ID,Codeshare,Stops,Equipment,Source latitude,Source longitude,Source Country,Destination latitude,Destination longitude,Destination Country,Distance
180,2P,897.0,BSO,6019.0,MNL,2397.0,NaN,0,DH4,20.45130,121.980003,Philippines,14.50860,121.019997,Philippines,665.557929
181,2P,897.0,BXU,4200.0,MNL,2397.0,NaN,0,320,8.95150,125.478800,Philippines,14.50860,121.019997,Philippines,783.536811
182,2P,897.0,CBO,2399.0,MNL,2397.0,NaN,0,320,7.16524,124.209999,Philippines,14.50860,121.019997,Philippines,883.909695
183,2P,897.0,CGY,2400.0,MNL,2397.0,NaN,0,320,8.41562,124.611000,Philippines,14.50860,121.019997,Philippines,779.513546
184,2P,897.0,CRM,6024.0,MNL,2397.0,NaN,0,DH4,12.50240,124.636002,Philippines,14.50860,121.019997,Philippines,449.993165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66286,Z2,9764.0,MNL,2397.0,TAG,3994.0,NaN,0,320 319,14.50860,121.019997,Philippines,9.66408,123.852997,Philippines,618.278096
66287,Z2,9764.0,MYY,3266.0,MNL,2397.0,NaN,0,320,4.32201,113.987000,Malaysia,14.50860,121.019997,Philippines,1365.431535
66288,Z2,9764.0,PPS,2433.0,MNL,2397.0,NaN,0,320 319,9.74212,118.759003,Philippines,14.50860,121.019997,Philippines,581.861803
66291,Z2,9764.0,TAC,2422.0,MNL,2397.0,NaN,0,320,11.22760,125.028000,Philippines,14.50860,121.019997,Philippines,566.519322


In [49]:
#we can add attributes here
G_Asean = nx.from_pandas_edgelist(routes_asean, 'Source airport', 'Destination airport', ['Distance'])

In [52]:
deg_asean = dict(G_Asean.degree())
deg_asean['RGN']

18

In [72]:
totaldf[totaldf['APT'].isin(air_asean)].sort_values(by='Deg',ascending=False)

,APT,Deg,BET
159,SIN,123,0.031541
158,BKK,120,0.023882
167,KUL,110,0.018872
79,MNL,77,0.020630
545,CGK,63,0.017646
...,...,...,...
2177,IPH,1,0.000000
2105,NAW,1,0.000000
1922,UNN,1,0.000000
1920,TST,1,0.000000
